In [ ]:
import pandas as pd
import random
import copy
from collections import Counter

In [ ]:
output_filename = ''
input_filename = ''

In [ ]:
# 이름 통일 시킨다.
def change_A(time_list):
  result = []
  for time in time_list:
    if time == '10시':
      result.append('오전 10시')
    else:
      result.append(time)
  return result

In [ ]:
# 파일 -> 결과정보, 시간정보, 사람정보, 평균인원수
def 파일가져오기(파일이름):
  df = pd.read_excel(파일이름)
  df = df[~df.duplicated(subset='성함', keep='last')]
  print("column_names: ",df.columns.tolist())
  df = df.rename(columns={'가능 시간 ': '가능 시간'})
  df['가능 시간'] = df['가능 시간'].str.split(', ')
  df['가능 시간'] = df['가능 시간'].apply(change_A)
  df_exploded = df.explode('가능 시간')[['가능 시간','성함']]
  unique_times = df_exploded['가능 시간'].unique()

  df_result = pd.DataFrame()
  df_result['가능 시간'] = unique_times
  df_result['성함'] = [[] for _ in range(len(df_result))]
  result_dict = df_result.set_index('가능 시간')['성함'].to_dict()

  time_df = df_exploded.groupby('가능 시간').agg(lambda x: x.unique().tolist() if x.dtype == 'O' else x.tolist()).reset_index()
  time_dict = time_df.set_index('가능 시간')['성함'].to_dict()

  people_dict = df.set_index('성함')['가능 시간'].to_dict()
  average_number = int(df.shape[0]/df_result.shape[0])

  return result_dict,time_dict,people_dict,average_number

In [ ]:
def pop_person(name):
  for key in time_dict:
    if name in time_dict[key]:
      time_dict[key].remove(name)
  if name in people_dict:
    people_dict.pop(name)

def pop_time(time):
  for key in people_dict:
    if time in people_dict[key]:
      people_dict[key].remove(time)
  if time in time_dict:
    time_dict.pop(time)

In [ ]:
def 사람과시간제거하기(사람들,시간들):
  time_excuted = False
  person_excuted = False
  if len(시간들) > 0:
    time_excuted = True
    for time in 시간들:
      pop_time(time)
  if len(사람들) > 0:
    person_excuted = True
    for name in 사람들:
      pop_person(name)
  if(time_excuted and len(time_dict) != 1):
    어쩔수없는사람배정()
    print("어쩔수없는사람배정() people_dict: ",len(people_dict)," time_dict: ",len(time_dict))
  if(person_excuted):
    어쩔수없는시간배정()
    print("어쩔수없는시간배정() people_dict: ",len(people_dict)," time_dict: ",len(time_dict))

# 오류
def 어쩔수없는사람배정():
  # 어쩔수 없는 사람배정
  배정완료된사람들 = []
  배정완료된시간들 = []
  for key in people_dict: # RuntimeError: dictionary changed size during iteration
    if(len(people_dict[key]) == 1):
      time = people_dict[key][0]
      result_dict[time].append(key)
      배정완료된사람들.append(key)
      if(len(result_dict[time]) >= average_number):
        배정완료된시간들.append(time)
  사람과시간제거하기(배정완료된사람들,배정완료된시간들)


def 어쩔수없는시간배정():
  # 어쩔수 없는 시간배정
  배정완료된사람들 = []
  배정완료된시간들 = []
  for key in time_dict:
    # 남아있는 카드를 모두 써야 할때
    if(len(time_dict[key]) <= (average_number - len(result_dict[key]))):
      print('어쩔수없는시간배정 실행', key,": ",time_dict[key])
      result_dict[key].extend(time_dict[key])
      배정완료된사람들.extend(time_dict[key])
      배정완료된시간들.append(key)

  # Use Counter to find duplicated values
  counter = Counter(배정완료된사람들)
  # Get duplicated values
  duplicates = [item for item, count in counter.items() if count > 1]
  if len(duplicates) > 0:
    print("중복배정된 사람들: ", duplicates)

  # Get unique values
  배정완료된사람들 = list(counter.keys())

  사람과시간제거하기(배정완료된사람들,배정완료된시간들)

In [ ]:
def sort_dict(my_dict):
  sorted_keys = sorted(my_dict, key=lambda k: len(my_dict[k]))
  return sorted_keys

def 랜덤시간배정(time):
  배정완료된사람들 = []
  배정완료된시간들 = []
  names = random.sample(time_dict[time], average_number - len(result_dict[time]))
  result_dict[time].extend(names)
  배정완료된시간들.append(time)
  배정완료된사람들.extend(names)
  사람과시간제거하기(배정완료된사람들,배정완료된시간들)

def 남은사람배정(data):
  print('---------------------------')
  print("남은사람들 배정: ",len(data),"명")
  already_assignmed_time = []
  for name in data:
    times = list(set(data[name])- set(already_assignmed_time))
    if(len(times) != 0):
      random_time = random.choice(times)
      already_assignmed_time.append(random_time)
    else:
      random_time = random.choice(data[name])
    print(name,': ',random_time)
    result_dict[random_time].append(name)
  print("-----------배정완료------------")

def 총사람수(result_dict):
  total = 0
  for a in result_dict:
    total += len(result_dict[a])
  print('결과 총사람수: ',total)
  print('남은 사람 총사람수: ', len(people_dict))


def 배정하기(file_name):
  deep_copy = copy.deepcopy(people_dict)
  print('---------- #0 -----------')
  어쩔수없는사람배정()
  어쩔수없는시간배정()
  총사람수(result_dict)
  for i in range(len(time_dict)):
    print('-----------#{}------------'.format(i+1))
    랜덤시간배정(sort_dict(time_dict)[0])
    총사람수(result_dict)
    if(len(time_dict) == 0):
      break

  remained_people_dict = {key: deep_copy[key] for key in list(people_dict.keys())}
  남은사람배정(remained_people_dict)

  for time in result_dict:
    print(time , ": ",len(result_dict[time]),'명 -',result_dict[time])
  pd.DataFrame.from_dict(result_dict, orient='index').to_excel(file_name)


In [ ]:
# 다시배정하고 싶으면 밑에 두개 실행

In [ ]:
result_dict,time_dict,people_dict,average_number = 파일가져오기(input_filename)

In [ ]:
배정하기(output_filename)